# All LeetCode SQL exercises solved! (sometimes multiple times!)

1. Second Highest Salary in Employees Database

In [ ]:
SELECT IFNULL(
    (SELECT DISTINCT Salary
    FROM Employee
    ORDER BY Salary DESC
    LIMIT 1 OFFSET 1), NULL) AS SecondHighestSalary
## or ##
SELECT IFNULL(
    (SELECT MAX(Salary)
FROM Employee
WHERE Salary < (SELECT MAX(Salary) FROM Employee)), NULL) AS SecondHighestSalary

2. Nth Highest Salary in Employees Database

In [ ]:
CREATE FUNCTION getNthHighestSalary(N INT) RETURNS INT
BEGIN

DECLARE M INT;
SET M = N - 1;
  RETURN (
      SELECT IFNULL(
      (SELECT DISTINCT SALARY
      FROM Employee
      ORDER BY Salary DESC
      LIMIT 1 OFFSET M), 
      NULL) AS NthHighestSalary
  );
END

3. Rank Scores from a table

In [ ]:
SELECT c.Score, Rank
FROM Scores as c
INNER JOIN (
    SELECT SCORE, @count:= @count+1 AS RANK
    FROM (
        SELECT DISTINCT Score FROM Scores
        ORDER BY Score DESC) AS B,
        (SELECT @count:=0) AS A) AS d
ON c.Score = d.Score
ORDER BY Score DESC

4. Output Consecutive Numbers

In [ ]:
SELECT DISTINCT logs1.Num AS ConsecutiveNums # select unique numbers in table
FROM Logs AS logs1, Logs AS logs2, Logs AS logs3 # 3 times from the same table with a different alias
WHERE logs1.Id = logs2.Id-1 # I want the same id happening 3 times consecutively, I am joining them as a chain
AND logs2.Id = logs3.Id-1
AND logs1.Num = logs2.Num # and I want the number to be the same in all three cases
AND logs2.Num = logs3.Num

5. Employees Earning More than their Managers

In [ ]:
Select employees.Name AS 'Employee'
FROM Employee as employees, Employee as managers #We need the information twice!
WHERE employees.ManagerId = managers.Id
AND employees.Salary > managers.Salary #notice how we can compare the salaries directly inside the WHERE statement

6. Find Duplicate Emails

In [ ]:
SELECT DISTINCT p1.Email
FROM Person AS p1, Person AS p2
WHERE p1.Email = p2.Email
AND p1.id != p2.id
## or ##
SELECT Email
FROM Person
GROUP BY Email
HAVING COUNT(Email) > 1

7. Identifying Customers who never Order

In [ ]:
SELECT Name AS Customers
FROM Customers AS c
WHERE c.Id NOT IN (  ## notice how we use a subquery in order to create the numbers where Id will be NOT IN  
    SELECT CustomerId
    FROM Orders);
## or ##
SELECT Name AS Customers
FROM Customers
LEFT JOIN Orders
ON Customers.Id = Orders.CustomerId  ## or we can create a table and select those rows with missing values (which satisfies
WHERE Orders.CustomerId IS NULL      ## our criterion). This solution is a bit slower

8. Department Top Three Salaries

In [ ]:
SELECT d.Name AS Department, e.Name AS Employee, e.Salary
FROM Department d
JOIN Employee e
WHERE e.DepartmentId=d.Id ## here I'm just joining the two tables, but then I add another condition:
AND (
    SELECT COUNT(DISTINCT Salary)
    FROM Employee
    WHERE DepartmentId=d.Id
    AND Salary > e.Salary) < 3 ## for each salary which is among Top 3, it must be less than 
                               ## or equal to at most 3 other salaries in that department
    ## or ##
SELECT x.Name AS Department, a.Name AS Employee, a.Salary AS Salary
FROM Employee a
JOIN (SELECT DISTINCT DepartmentId, Salary FROM Employee) b
ON a.DepartmentId = b.DepartmentId
AND a.Salary <= b.Salary
JOIN Department x
ON a.DepartmentId = x.Id
GROUP BY  a.DepartmentId, a.Name, a.Salary
HAVING COUNT(a.Salary) <= 3;

9. Delete Duplicate Emails 

In [ ]:
DELETE FROM Person           ## I'm telling SQL to delete all rows that are not found in the 'table of minimum ids' 
WHERE Id NOT IN              ## that I create below in a subquery.
    (SELECT *                ## here I just select everything from the table of the subquery below
    FROM (
         SELECT MIN(Id) 
         FROM Person 
         GROUP BY Email) p); ## in this subquery I select the minimum id for each email in Person
    ## or ##
DELETE p1 FROM Person p1,    
    Person p2
WHERE
    p1.Email = p2.Email AND p1.Id > p2.Id

10. Rising Temperatue

In [ ]:
SELECT t1.Id
FROM Weather AS t1, Weather AS t2  ## remember that you can select from many tables with the FROM clause!
WHERE t1.Temperature > t2.Temperature          ## Our two conditions is temperature should be greater and the difference
AND DATEDIFF(t1.RecordDate, t2.RecordDate)=1;  ## should be one day  

11. Trips and Users

In [ ]:
SELECT Request_at AS Day,
    ROUND(SUM(CASE WHEN Status LIKE 'cancelled_%' THEN 1 ELSE 0 END)/COUNT(*),2) AS 'Cancellation Rate' ## conditional inside SELECT
FROM Trips
WHERE Client_Id IN (SELECT Users_Id FROM Users WHERE Banned = 'No') ## I'm making two subqueries where the tables are already
AND Driver_Id IN (SELECT Users_Id FROM Users WHERE Banned = 'No')   ## filtered for only the results that are acceptable
AND Request_at BETWEEN '2013-10-01' AND '2013-10-03'
GROUP BY Request_at;                                                ## Since I have an aggregation above, I need to GB 

12. Big Countries

In [ ]:
SELECT name, population, area
FROM World
WHERE population > 25000000
OR area > 3000000

13. Classes with More than 5 Students

In [ ]:
SELECT class
FROM (
    SELECT class, COUNT(DISTINCT student) AS num  ## I use this subquery so I can erase the extra column from my final result
    FROM courses
    GROUP BY class
    ) temp
WHERE num >= 5 ## note that I can use the alias num because the other table was already created in the subquery; otherwise I 
               ## can't

 ## or ##
    
SELECT class
FROM courses
GROUP BY class
HAVING COUNT(DISTINCT student) >= 5

14. Game Play Analysis I

In [ ]:
SELECT player_id, MIN(event_date) AS first_login
FROM Activity
GROUP BY player_id;

15. Game Play Analysis II

In [ ]:
SELECT player_id, device_id 
FROM activity 
WHERE (player_id, event_date) IN (   ## the variable to which you apply your WHERE clause can be a tuple!
                                SELECT player_id, MIN(event_date)
                                FROM activity 
                                GROUP BY by player_id
                                 ) 

16. Game Play Analysis III

In [ ]:
SELECT a1.player_id, a1.event_date, SUM(a2.games_played) AS games_played_so_far
FROM Activity a1, Activity a2  ## self join
WHERE a1.event_date >= a2.event_date
AND a1.player_id = a2.player_id
GROUP BY a1.player_id, a1.event_date; ## If I don't group by player and event date here, I'll just get the total sum of 33

  ## or ##
    
SELECT a1.player_id, a1.event_date,
 (SELECT sum(a2.games_played) FROM activity a2
 WHERE a1.player_id = a2.player_id AND a1.event_date >=a2.event_date
 GROUP BY a1.player_id)
 AS games_played_so_far
 FROM activity a1;  ## this attempt is using a correlated subquery and it is much slower than the one above

17. Game Play Analysis IV

In [ ]:
SELECT ROUND(COUNT(t2.player_id)/COUNT(t1.player_id),2) AS fraction
FROM (
    SELECT player_id, MIN(event_date) AS first_login
    FROM Activity 
    GROUP BY player_id) t1 
LEFT JOIN Activity t2
ON t1.player_id = t2.player_id AND t1.first_login = t2.event_date - 1

    ## or ## 
    
SELECT ROUND(SUM(CASE WHEN temp.min_date + 1 = a.event_date THEN 1 ELSE 0 END) / COUNT(DISTINCT temp.player_id), 2) AS fraction
FROM (
    SELECT player_id, MIN(event_date) AS min_date   ## this is a much faster solution
    FROM Activity
    GROUP BY player_id) temp
JOIN Activity a
ON a.player_id = temp.player_id;

18. Employee Bonus

In [ ]:
SELECT name, bonus
FROM Employee e
LEFT JOIN Bonus b
ON b.empId = e.empId
WHERE bonus < 1000
OR bonus IS NULL 

19. Find Customer Referee

In [ ]:
SELECT name
FROM customer
WHERE referee_id != 2
OR referee_id IS NULL

20. Project Employees I

In [ ]:
SELECT project_id, ROUND(AVG(experience_years),2) AS average_years
FROM Project p
LEFT JOIN Employee e
ON e.employee_id = p.employee_id
GROUP BY project_id;

21. Project Employees II

In [ ]:
SELECT project_id         ### ???????
FROM (
    SELECT project_id, COUNT(DISTINCT employee_id) AS num_employees
    FROM Project
    GROUP BY project_id
    ORDER BY num_employees DESC) temp
LIMIT 1

 ## or ##
SELECT project_id
FROM project
GROUP BY project_id
HAVING COUNT(employee_id) =  ## I can have an abstract equality here, equating a varible COUNT to a table that has aggregated
(                            ## data. Notice that all the aggregation, purging and ordering of the data is done inside the  
    SELECT count(employee_id)    ## subquery. What this subquery is giving me is the number of employees that the project
    FROM project                 ## with most employees has 
    GROUP BY project_id
    ORDER BY count(employee_id) desc
    LIMIT 1
)

22. Project Employees III

In [ ]:
SELECT p.project_id, e.employee_id
FROM Project p
JOIN Employee e 
ON e.employee_id = p.employee_id
WHERE (p.project_id, e.experience_years) IN (  ##we are doing an inner join where the project id and employee years match
    SELECT p.project_id, MAX(experience_years) ## the id and maximum experience years per project
    FROM project p
    JOIN Employee e
    ON e.employee_id=p.employee_id
    GROUP BY project_id)

## or ##

SELECT sub2.project_id, sub1.employee_id  ## notice here how both of our tables are subqueries
FROM (SELECT p.project_id, p.employee_id, e.experience_years
     FROM Project p 
     JOIN Employee e
     ON p.employee_id = e.employee_id) sub1 ## this table is the full table with all the data
JOIN (SELECT p.project_id, MAX(e.experience_years) AS max_exp  ## this is the table with each project id's and its maximum
            FROM Project p                                     ## experience years
            JOIN Employee e
            ON p.employee_id = e.employee_id
            GROUP BY p.project_id) sub2
ON sub1.project_id = sub2.project_id AND sub1.experience_years = sub2.max_exp ## finally we tell SQL to keep only those 
                                                                              ## project ids such that the experience years 
                                                                               ## match the maximum exp.

23. Actors and Directors Who Cooperated At Least Three Times

In [ ]:
SELECT actor_id, director_id
FROM (
    SELECT actor_id, director_id, COUNT(*) AS N
    FROM ActorDirector
    GROUP BY actor_id, director_id  ## on this exercise, the meaningful row entry is (actor_id, director_id)
    HAVING N > 2) temp
ORDER BY actor_id, director_id

24. Product Sales Analysis

In [ ]:
SELECT product_name, s.year, price
FROM Sales s
JOIN Product p
ON p.product_id = s.product_id

25. Product Sales Analysis II

In [ ]:
SELECT product_id, SUM(quantity) AS total_quantity
FROM Sales
GROUP BY product_id;

26. Consecutive Available Seats

In [ ]:
SELECT DISTINCT a.seat_id
FROM cinema a JOIN cinema b  ## we perform a self join which by itself would return the cartesian product of the tables
ON ABS(a.seat_id - b.seat_id) = 1   ## but then we constrain it such that the absolute diff between seat_ids is 1
WHERE a.free = true AND b.free = true ## and then the conditions such that both seats are free
ORDER BY a.seat_id;

## or ##

SELECT DISTINCT seat_id      
FROM cinema
WHERE free = true         ## select all ids that are free
AND (             
    seat_id - 1 IN (      ## and also such that the previous seat was also included in the 'table of free seats'
         SELECT seat_id
        FROM cinema
        WHERE free = true) OR
    seat_id + 1 IN (     ## OR such that the next seat is also included in the 'table of free seats'
        SELECT seat_id
        FROM cinema
        WHERE free = true))

27. Sales Person

In [ ]:
SELECT name
FROM salesperson
WHERE name NOT IN (    ## I'm telling SQL to grab the people who have not sold to red
    SELECT s.name      ## this is a subquery that creates a table of all the people who have sold to RED company
    FROM salesperson s
    JOIN orders o
    ON o.sales_id = s.sales_id
    JOIN company c
    ON c.com_id = o.com_id
    WHERE c.name = 'RED')

28. Reported Posts

In [ ]:
SELECT extra AS report_reason, COUNT(DISTINCT post_id) AS report_count
FROM Actions
WHERE action = 'report'
AND extra IS NOT NULL
AND action_date = '2019-07-04'
GROUP BY extra;

29. Managers with at least 5 Direct Reports

In [ ]:
SELECT Name
FROM Employee
WHERE Id IN (   ## we want the Id to be the one such that it appears as a 'ManagerId' at least 5 times
    SELECT ManagerId
    FROM Employee
    GROUP BY ManagerId
    HAVING COUNT(ManagerId) >= 5)

30. Winning Candidate

In [ ]:
## I can solve this exercise using a WHERE clause with a subquery or joining tables. Turns out the WHERE method is much 
## more efficient. 

SELECT Name
FROM Candidate
WHERE id = (   ## From this subquery I am getting a single number, so the WHERE condition holds.
    SELECT CandidateId
    FROM Vote
    GROUP BY CandidateId
    ORDER BY COUNT(*) DESC
    LIMIT 1)

## or ##

SELECT Name
FROM Candidate
JOIN (         ## Here I am creating the table of candidate ids ordered by number of votes received
    SELECT CandidateId    ## it turns out to be just the winner, because I limit the output to only the candidate with 
    FROM Vote             ## maximum votes.
    GROUP BY CandidateId
    ORDER BY COUNT(*) DESC
    LIMIT 1) winner
ON Candidate.id = winner.CandidateId

31. Customer Placing the Largest Number of Orders

In [ ]:
SELECT customer_number
FROM orders
GROUP BY customer_number
ORDER BY COUNT(*) DESC
LIMIT 1

32. Friend Requests II: Who Has the Most Friends?

In [ ]:
SELECT requester_id AS id, COUNT(requester_id) AS num
FROM (
    SELECT requester_id 
    FROM request_accepted
    UNION ALL
    SELECT accepter_id
    FROM request_accepted) ids
GROUP BY requester_id
ORDER BY 2 DESC
LIMIT 1;

33. Not Boring Movies

In [ ]:
SELECT *
FROM cinema
WHERE ID%2 <> 0
AND description <> 'boring'
ORDER BY rating DESC;

34. Immediate Food Delivery

In [ ]:
SELECT ROUND(100*AVG(order_date = customer_pref_delivery_date), 2) AS immediate_percentage
FROM Delivery;

## or ##

SELECT ROUND(100*SUM(CASE 
                       WHEN order_date = customer_pref_delivery_date THEN 1 
                       ELSE 0 END)/COUNT(order_date),2) AS immediate_percentage
FROM Delivery;

35. Count Student Numbers in Department

SELECT dept_name, COUNT(student_id) AS student_number
FROM student s
RIGHT JOIN department d
ON d.dept_id = s.dept_id
GROUP BY dept_name
ORDER BY 2 DESC, 1;

36. Unpopular Books

In [ ]:
SELECT  b.book_id, name
FROM Books AS b
LEFT JOIN
Orders AS o
ON b.book_id = o.book_id
AND datediff('2019-06-23', dispatch_date) <=365
WHERE datediff('2019-06-23', available_from)>=30
GROUP BY b.book_id
HAVING SUM(quantity)<10 OR SUM(quantity) is null

37. Sales Analysis I

In [ ]:
SELECT seller_id
FROM Sales
GROUP BY seller_id
HAVING SUM(price) = (SELECT SUM(price) AS total
                    FROM Sales
                    GROUP BY seller_id
                    ORDER BY 1 DESC
                    LIMIT 1); 

38. Sales Analysis II

39. Sales Analysis III

In [ ]:
SELECT product_id, product_name
FROM Product
WHERE product_id NOT IN (SELECT product_id 
                         FROM Sales 
                         WHERE sale_date NOT BETWEEN '2019-01-01' AND '2019-03-31');

40. User Activity for the past 30 Days

In [ ]:
SELECT activity_date AS day, COUNT(DISTINCT user_id) AS active_users
FROM Activity
WHERE activity_date BETWEEN "2019-06-28" AND "2019-07-27"
GROUP BY 1

41. User Activity for the past 30 Days II

In [ ]:
SELECT IFNULL(ROUND(AVG(sessions),2),0.00) AS average_sessions_per_user
FROM (
    SELECT user_id, COUNT(DISTINCT session_id) AS sessions
    FROM Activity
    WHERE activity_date BETWEEN "2019-06-28" AND "2019-07-27"
    GROUP BY user_id) temp

42. Article Views II

In [ ]:
SELECT DISTINCT author_id AS id 
FROM Views 
WHERE author_id = viewer_id 
ORDER BY id

43. Second Degree Follower

In [ ]:
SELECT followee as follower, COUNT(distinct follower) as num
FROM follow
WHERE followee in (SELECT follower FROM follow)
GROUP BY 1